# sql_analysis.ipynb
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

This Jupyter Notebook analyzes the postgreSQL database.

The queries contained herin were developed in sql.sql and translated into sqlalchemy and pandas.

In [1]:
# Import Dependencies
import sys
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Import database password
# Note: user needs to update postgres_pswd.py file with their database information or
# point to a personal untracked file
from postgres_pswd import host, database, username, passwd
if host == 'YOUR DATABASE HOST HERE':
    sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
    from postgres_remote import host, database, username, passwd

# Create engine to mutual_funds database
engine_startup = 'postgresql://' + username + ":" + passwd + "@" + host + '/' + database
engine = create_engine(engine_startup)

In [2]:
# reflect the existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
Base.classes.keys()

['sp500', 'fund_holdings']

In [3]:
# Save references to each table
sp500 = Base.classes.sp500
holdings = Base.classes.fund_holdings

In [4]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [5]:
# Collect Total Market Value for each fund
result = session.query(\
    holdings.fund_name, func.sum(holdings.market_value)).\
    group_by(holdings.fund_name).\
    all()

# Build Pandas DataFrame
holdings_fund_name = []
holdings_market_value = []
for row in result:
    (fund_name, market_value) = row
    holdings_fund_name.append(fund_name)
    holdings_market_value.append(market_value)

market_total_df = pd.DataFrame({'fund_name':holdings_fund_name,'total_market_value':holdings_market_value})

market_total_df[['total_market_value']] = market_total_df[['total_market_value']].apply(pd.to_numeric)

print(market_total_df.info())
market_total_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fund_name           5 non-null      object 
 1   total_market_value  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
None


,fund_name,total_market_value
0,Voya LargeCap Growth Portfolio,6.532219e+09
1,Voya Russell LargeCap Index Portfolio,1.244100e+09
2,Voya U.S. Stock Index Portfolio,7.917050e+09
3,Voya LargeCap Value Portfolio,1.113767e+09
4,Voya Index Plus LargeCap Portfolio,9.599116e+08


In [6]:
# Collect SP 500 stocks for each fund
result = engine.execute(' \
    SELECT fund_holdings.fund_name, \
           fund_holdings.ticker, \
           sp500.gics_sector, \
           fund_holdings.market_value \
    FROM   sp500 \
    INNER  JOIN fund_holdings \
    ON     sp500.ticker = fund_holdings.ticker; \
    ')

# Build Pandas DataFrame
sp500_fund_name = []
sp500_ticker = []
sp500_gics_sector = []
sp500_market_value = []
for row in result:
    (fund_name, ticker, gics_sector, market_value) = row
    sp500_fund_name.append(fund_name)
    sp500_ticker.append(ticker)
    sp500_gics_sector.append(gics_sector)
    sp500_market_value.append(market_value)

sp500_holdings_df = pd.DataFrame({'fund_name':sp500_fund_name, 'ticker':sp500_ticker, 'gics_sector':sp500_gics_sector, 'market_value':sp500_market_value})
sp500_holdings_df[['market_value']] = sp500_holdings_df[['market_value']].apply(pd.to_numeric)

print(sp500_holdings_df.info())
sp500_holdings_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   fund_name     950 non-null    object 
 1   ticker        950 non-null    object 
 2   gics_sector   950 non-null    object 
 3   market_value  950 non-null    float64
dtypes: float64(1), object(3)
memory usage: 29.8+ KB
None


,fund_name,ticker,gics_sector,market_value
0,Voya Index Plus LargeCap Portfolio,ABT,Health Care,5016702.24
1,Voya Index Plus LargeCap Portfolio,ABBV,Health Care,2232899.00
2,Voya Index Plus LargeCap Portfolio,ACN,Information Technology,1238461.87
3,Voya Index Plus LargeCap Portfolio,ADBE,Information Technology,6519968.84
4,Voya Index Plus LargeCap Portfolio,AMD,Information Technology,606273.36


In [7]:
# Collect Total SP500 Market Value for each fund
sp500_market_total = sp500_holdings_df.groupby('fund_name').sum()
sp500_market_total = sp500_market_total.rename(columns={'market_value':'sp500_market_value'})
sp500_market_total

,sp500_market_value
fund_name,
Voya Index Plus LargeCap Portfolio,8.036399e+08
Voya LargeCap Growth Portfolio,5.540274e+09
Voya LargeCap Value Portfolio,9.717242e+08
Voya Russell LargeCap Index Portfolio,1.197636e+09
Voya U.S. Stock Index Portfolio,7.712523e+09


In [8]:
# Calculate percentage of SP 500 stock holdings

# Join market_value and sp_market_value analyses
holdings_analysis_df = market_total_df.join(sp500_market_total, on='fund_name',how='left')

# Calculate SP 500 holdings percentage 
holdings_analysis_df['sp500_percentage'] = round(100*holdings_analysis_df['sp500_market_value']/holdings_analysis_df['total_market_value'],2)
holdings_analysis_df = holdings_analysis_df.sort_values('sp500_percentage',ascending=False)

# Convert market totals into millions
holdings_analysis_df['total_market_value_MM'] = round(holdings_analysis_df['total_market_value']/1000000,3)
holdings_analysis_df['sp500_market_value_MM'] = round(holdings_analysis_df['sp500_market_value']/1000000,3)

# Remove unnecessary columns
holdings_analysis_df = holdings_analysis_df[['fund_name','total_market_value_MM','sp500_market_value_MM','sp500_percentage']]

# Set fund_name as index
holdings_analysis_df = holdings_analysis_df.set_index('fund_name')
holdings_analysis_df

,total_market_value_MM,sp500_market_value_MM,sp500_percentage
fund_name,,,
Voya U.S. Stock Index Portfolio,7917.050,7712.523,97.42
Voya Russell LargeCap Index Portfolio,1244.100,1197.636,96.27
Voya LargeCap Value Portfolio,1113.767,971.724,87.25
Voya LargeCap Growth Portfolio,6532.219,5540.274,84.81
Voya Index Plus LargeCap Portfolio,959.912,803.640,83.72


In [9]:
# Collect top 3 Sectors for SP 500 stocks for each fund

# Collect Total Sector Value by fund
sp500_sector_weight_df = sp500_holdings_df.groupby(['fund_name','gics_sector']).sum()
sp500_sector_weight_df = sp500_sector_weight_df.rename(columns={'market_value':'sector_market_value'})

# Sort funds by sector_market_value
sp500_sector_weight_group_df = sp500_sector_weight_df['sector_market_value'].groupby('fund_name',group_keys=False)
sp500_sector_weight_sort_df = sp500_sector_weight_group_df.apply(lambda x: x.sort_values(ascending=False).head(3))
sp500_sector_weight_df = sp500_sector_weight_sort_df.to_frame()

# Get total SP 500 Market Value for comparison
sp500_market_value_list = []
for row in sp500_sector_weight_df.iterrows():
    sp500_market_value_list.append(sp500_market_total.loc[row[0][0],'sp500_market_value'])
sp500_sector_weight_df['sp500_market_value'] = sp500_market_value_list

# Calculate SP500 sector percentage
sp500_sector_weight_df['sector_percentage'] = round(100*sp500_sector_weight_df['sector_market_value']/sp500_sector_weight_df['sp500_market_value'],1)

# Convert sector_market_value into millions
sp500_sector_weight_df['sector_market_value_MM'] = round(sp500_sector_weight_df['sector_market_value']/1000000,3)

# Collect relevant columns
sp500_sector_weight_df = sp500_sector_weight_df[['sector_market_value_MM','sector_percentage']]

sp500_sector_weight_df

sector_market_value_MM  \
fund_name                             gics_sector                                      
Voya Index Plus LargeCap Portfolio    Information Technology                 213.743   
                                      Health Care                            119.579   
                                      Consumer Discretionary                 105.477   
Voya LargeCap Growth Portfolio        Information Technology                2513.144   
                                      Consumer Discretionary                1099.587   
                                      Communication Services                 593.484   
Voya LargeCap Value Portfolio         Financials                             223.498   
                                      Health Care                            137.534   
                                      Communication Services                 101.679   
Voya Russell LargeCap Index Portfolio Information Technology                 354.840   
                                      Health Care                            163.470   
                                      Communication Services                 159.930   
Voya U.S. Stock Index Portfolio       Information Technology                2091.261   
                                      Health Care                           1002.662   
                                      Consumer Discretionary                 984.948   

                                                              sector_percentage  
fund_name                             gics_sector                                
Voya Index Plus LargeCap Portfolio    Information Technology               26.6  
                                      Health Care                          14.9  
                                      Consumer Discretionary               13.1  
Voya LargeCap Growth Portfolio        Information Technology               45.4  
                                      Consumer Discretionary               19.8  
                                      Communication Services               10.7  
Voya LargeCap Value Portfolio         Financials                           23.0  
                                      Health Care                          14.2  
                                      Communication Services               10.5  
Voya Russell LargeCap Index Portfolio Information Technology               29.6  
                                      Health Care                          13.6  
                                      Communication Services               13.4  
Voya U.S. Stock Index Portfolio       Information Technology               27.1  
                                      Health Care                          13.0  
                                      Consumer Discretionary               12.8

In [10]:
# Close Session
session.close()